## DF NESESARIOS 

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder



In [3]:
df = pd.read_csv('merged.csv')
df_sin_NaN = df.dropna(subset=['category'])
# Filtrar las filas donde la columna "category" contiene la palabra "restaurant"
df_restaurantes = df_sin_NaN[df_sin_NaN['category'].str.contains('restaurant', case=False)]
# CREACION DE MUESTRAs
# Muestra ml  
muestra= df.sample(4000)


FileNotFoundError: [Errno 2] No such file or directory: 'merged.csv'

### MODELO DE RECOMENDACION DE RESTAURANT EN BASE A SENTIMENT SCORE Y AVG_RATING 

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

df_restaurantes.reset_index(drop=True, inplace=True)


# Importar MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

# Inicializar el escalador MinMaxScaler
scaler = MinMaxScaler()

# Normalizar los sentimiento scores
df_restaurantes['sentiment_score_normalized'] = scaler.fit_transform(df_restaurantes['sentiment_analysis'].values.reshape(-1, 1))
# Normaliza la nueva variable si es necesario
df_restaurantes['avg_rating_normalized'] = (muestra_restaurantes['avg_rating'] - df_restaurantes['avg_rating'].mean()) / df_restaurantes['avg_rating'].std()

# Combina la nueva variable con los puntajes de sentimiento existentes
matriz_caracteristicas = np.concatenate([df_restaurantes['sentiment_score_normalized'].values.reshape(-1, 1),
                                        df_restaurantes['avg_rating_normalized'].values.reshape(-1, 1)], axis=1)

# Calcula la similitud del coseno entre las columnas de la matriz de características
cosine_similarity_sentiment= cosine_similarity(matriz_caracteristicas)

def recomendacion_sentimiento(id: int):
    if id not in df_restaurantes['business_id'].values:
        return {'mensaje': 'No existe el id restaurante.'}
    
    # Filtrar el DataFrame para excluir el restaurante con el ID proporcionado
    filtered_df = df_restaurantes[df_restaurantes['business_id'] != id]
    
    # Obtener el índice del restaurante con el ID proporcionado
    idx = df_restaurantes[df_restaurantes['business_id'] == id].index[0]
    
    # Calcular la similitud del coseno entre los sentimiento scores del restaurante proporcionado y los demás
    sim_scores = sorted(list(enumerate(cosine_similarity_sentiment[idx])), key=lambda x: x[1], reverse=True)
    
    # Seleccionar los 5 restaurantes más similares (excluyendo el proporcionado)
    sim_ind = [i for i, _ in sim_scores[:5]]
    
    # Resetear los índices del DataFrame filtrado
    filtered_df.reset_index(drop=True, inplace=True)
    
    # Obtener los IDs de los restaurantes recomendados
    sim_restaurants = filtered_df.loc[sim_ind, 'business_id'].values.tolist()
    
    return {'restaurantes recomendados': sim_restaurants}



KeyboardInterrupt: 

In [8]:
def recomendacion_sentimiento_category(id: int):
    if id not in set(muestra['category']):
        return {'mensaje': 'No existe la categoría proporcionada.'}
    
    # Filtrar el DataFrame para seleccionar solo las filas con la categoría deseada
    filtered_df = muestra[muestra['category'] == id]
    
    # Obtener el índice del primer 
    idx = filtered_df.index[0]
    
    # Calcular la similitud del coseno entre los sentimiento scores de la category proporcionado y los demás
    sim_scores = sorted(list(enumerate(cosine_similarity_sentiment[idx])), key=lambda x: x[1], reverse=True)
    
    # Seleccionar los  más similares (excluyendo el proporcionado)
    sim_ind = [i for i, _ in sim_scores[:5]]
    
    # Obtener los IDs de los recomendados
    sim_restaurants = filtered_df.loc[sim_ind, 'business_id'].values.tolist()
    
    return {'recomendados': sim_restaurants}

## Modelos de prediccion 

JUSTIFICACION 
cree estos modelos de ML para : 
1 Poder dar recomendaciones de negocios que no tengan reseña prediciendo su avg_rating y su sentiment score (Esto permitira brindar informacion importante al usuari y uba mejor experiencia ya que si desea consultar informacion de un restaurante nuevo por ejemplo este no le sera imposible por falta de datos )
2 Poder simular la creacion de un nuevo negocio y en base a sus recomendaciones basadas en sus datos de avg y sentiment score predichos estudiar a su competencia e identificar tendencias 


## PREDICCION DE AVG_RETING

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

# Suponiendo que 'muestra' es tu conjunto de datos

# Selecciona características relevantes
features = ['latitude', 'longitude', 'category']

# Codificar la variable de categoría
encoder = OneHotEncoder(drop='first')  # drop='first' para evitar la colinealidad
category_encoded = encoder.fit_transform(muestra[['category']])

# Combinar características originales con las codificadas
X = np.hstack((muestra[['latitude', 'longitude']].values, category_encoded.toarray()))

# Obtener la variable objetivo
y = muestra['avg_rating']

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
model = LinearRegression()
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
predictions = model.predict(X_test)

# Evaluar el rendimiento del modelo
mse = mean_squared_error(y_test, predictions)
print("Error cuadrático medio (MSE):", mse)




Error cuadrático medio (MSE): 7.024668771093482e+21


### UTILIZACION DEL MODELO ENTRENADO 

In [19]:
# Supongamos que queremos hacer una predicción para una ubicación en Florida
# Utilizaremos una latitud y longitud cercanas a Orlando, Florida
latitud = 28.5383
longitud = -81.3792

# También proporcionaremos una categoría de negocio, por ejemplo, "Restaurantes"
categoria = "Apartment building"

# One-hot encode la categoría utilizando el mismo encoder que utilizamos durante el entrenamiento
categoria_encoded = encoder.transform([[categoria]]).toarray() # Aquí usamos el método toarray()

# Asegura que categoria_encoded sea bidimensional
if categoria_encoded.ndim == 1:
    categoria_encoded = categoria_encoded.reshape(1, -1)

# Asegura que latitud y longitud sean bidimensionales
latitud = np.array([[latitud]])
longitud = np.array([[longitud]])

# Combina las características de latitud, longitud y categoría
nuevo_negocio_features = np.concatenate((latitud, longitud, categoria_encoded), axis=1)

# Realiza la predicción utilizando el modelo entrenado
prediccion_avg_rating = model.predict(nuevo_negocio_features)

print("Predicción del promedio de calificación para el nuevo negocio en Florida:", prediccion_avg_rating)





Predicción del promedio de calificación para el nuevo negocio en Florida: [3.71283722]


c:\Users\Nazareno\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


### modelo de prediccion de sentiment score para x sin reseñas 

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

# Selecciona características relevantes
features = ['latitude', 'longitude', 'category']

# One-hot encode la variable de categoría
encoder = OneHotEncoder() 
category_encoded = encoder.fit_transform(muestra[['category']]).toarray()  # Convertir a matriz densa

# Combina las características originales con las codificadas
X = np.hstack((muestra[['latitude', 'longitude']].values, category_encoded))

# Obtiene la variable objetivo
y = muestra['sentiment_analysis']  # Ahora usamos el puntaje de sentimiento como variable objetivo

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión logística multinomial
model_2 = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model_2.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
predictions = model_2.predict(X_test)

# Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)


Accuracy: 0.89


c:\Users\Nazareno\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [30]:


# Supongamos que queremos hacer una predicción para una ubicación en Florida
# Utilizaremos una latitud y longitud cercanas a Orlando, Florida
latitud = 28.5383
longitud = -81.3792

# También proporcionaremos una categoría de negocio, por ejemplo, "Restaurantes"
categoria = "Apartment building"

# One-hot encode la categoría utilizando el mismo encoder que utilizamos durante el entrenamiento
categoria_encoded = encoder.transform([[categoria]])

# Asegura que latitud y longitud sean bidimensionales
latitud = np.array([[latitud]])
longitud = np.array([[longitud]])

# Combina las características de latitud, longitud y categoría
nuevo_negocio_features = np.hstack((latitud, longitud, categoria_encoded.toarray()))

# Realiza la predicción utilizando el modelo entrenado
prediccion_ = model_2.predict(nuevo_negocio_features)

print("Predicción del promedio de calificación para el nuevo negocio en Florida:", prediccion_)


Predicción del promedio de calificación para el nuevo negocio en Florida: [2]


c:\Users\Nazareno\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
